In [1]:
import os
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from joblib import dump
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
for dirname, _, filenames in os.walk('.\dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

.\dataset\consumable\alternaria_013.jpg
.\dataset\consumable\alternaria_014.jpg
.\dataset\consumable\alternaria_032.jpg
.\dataset\consumable\alternaria_040.jpg
.\dataset\consumable\alternaria_045.jpg
.\dataset\consumable\alternaria_046.jpg
.\dataset\consumable\alternaria_056.jpg
.\dataset\consumable\alternaria_058.jpg
.\dataset\consumable\alternaria_069.jpg
.\dataset\consumable\alternaria_122.jpg
.\dataset\consumable\anthracnose_002.jpg
.\dataset\consumable\anthracnose_018.jpg
.\dataset\consumable\anthracnose_026.jpg
.\dataset\consumable\anthracnose_040.jpg
.\dataset\consumable\anthracnose_044.jpg
.\dataset\consumable\anthracnose_054.jpg
.\dataset\consumable\anthracnose_092.jpg
.\dataset\consumable\anthracnose_103.jpg
.\dataset\consumable\anthracnose_117.jpg
.\dataset\consumable\anthracnose_121.jpg
.\dataset\consumable\anthracnose_123.jpg
.\dataset\consumable\anthracnose_124.jpg
.\dataset\consumable\anthracnose_126.jpg
.\dataset\consumable\healthy_001.jpg
.\dataset\consumable\healthy_0

In [3]:
# Directory with our training healthy pictures
a_dir = os.path.join('./dataset/consumable')

# Directory with our training not healthy pictures
b_dir = os.path.join('./dataset/not consumable')

In [4]:
a_names = os.listdir(a_dir)
print(a_names[:10])
b_names = os.listdir(b_dir)
print(b_names[:10])

['alternaria_013.jpg', 'alternaria_014.jpg', 'alternaria_032.jpg', 'alternaria_040.jpg', 'alternaria_045.jpg', 'alternaria_046.jpg', 'alternaria_056.jpg', 'alternaria_058.jpg', 'alternaria_069.jpg', 'alternaria_122.jpg']


FileNotFoundError: [WinError 3] The system cannot find the path specified: './dataset/not_consumable'

In [ ]:
print('total healthy images:', len(os.listdir(a_dir)))
print('total non-healthy images:', len(os.listdir(b_dir)))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 10
ncols = 2

# Index for iterating over images
pic_index = 0

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
a_pix = [os.path.join(a_dir, fname)
                for fname in a_names[pic_index-8:pic_index]]
b_pix = [os.path.join(b_dir, fname)
                for fname in b_names[pic_index-8:pic_index]]

for i, img_path in enumerate(a_pix + b_pix):
  sp = plt.subplot(nrows, ncols, (i % (nrows * ncols)) + 1)
  sp.axis('Off')
  img = mpimg.imread(img_path)
  plt.imshow(img)

In [ ]:
data_dir = './dataset'
batch_size = 64
epochs = 30
input_shape = (300, 300, 3)
image_paths = []
labels = []

for category in os.listdir(data_dir):
    category_dir = os.path.join(data_dir, category)
    if os.path.isdir(category_dir):
        for image_filename in os.listdir(category_dir):
            if image_filename.endswith('.jpg'):
                image_path = os.path.join(category_dir, image_filename)
                image_paths.append(image_path)
                labels.append(category)
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [ ]:
len(train_image_paths),len(test_image_paths), len(train_labels), len(test_labels)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# Create a label encoder
label_encoder = LabelEncoder()

# Fit the encoder with your labels
# Assuming `labels` is your list of labels
label_encoder.fit(labels)

# Save the encoder
dump(label_encoder, 'label_encoder.joblib')

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': train_image_paths, 'label': train_labels}),
    x_col='image_path',
    y_col='label',
    target_size=(150,150),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': test_image_paths, 'label': test_labels}),
    x_col='image_path',
    y_col='label',
    target_size=(150,150),
    batch_size=64,
    class_mode='categorical'
)

In [ ]:
history = model.fit(train_generator, epochs=30, steps_per_epoch=8, validation_data = validation_generator, verbose = 1, validation_steps=3)

In [ ]:
# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation Loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
model.save('new_mango.h5')